# DataLabPro AI - Notebook 04: Evaluación e Interpretación de Resultados

Diagnóstico Médico por Imágenes con IA Tradicional

Notebook 04: Evaluación e Interpretación

✅ Evaluación exhaustiva en conjunto de test

✅ Métricas clínicas detalladas

✅ Interpretabilidad con Grad-CAM

✅ Análisis de casos difíciles

✅ Reporte clínico automatizado

----------

In [ ]:
# Montar drive

from google.colab import drive
import os

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""
DataLabPro AI - Notebook 04: Evaluación e Interpretación de Resultados
Diagnóstico Médico por Imágenes con IA Tradicional

Autor: DataLabPro Team
Fecha: 2025
Objetivo: Evaluar rendimiento del mejor modelo y generar interpretaciones clínicas
"""

#@title ## 📊 Visualizaciones de Resultados - FUNCIONES FALTANTES Y CORRECCIONES

def create_confusion_matrix_plot(y_true, y_pred, class_names):
    """Crear matriz de confusión interactiva"""

    cm = confusion_matrix(y_true, y_pred)

    # Normalizar matriz de confusión
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Crear figura con subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Matriz de Confusión (Conteos)', 'Matriz de Confusión (Normalizada)'],
        specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}]]
    )

    # Matriz de confusión con conteos
    fig.add_trace(
        go.Heatmap(
            z=cm,
            x=class_names,
            y=class_names,
            colorscale='Blues',
            text=cm,
            texttemplate="%{text}",
            textfont={"size": 12},
            showscale=True,
            colorbar=dict(x=0.45),
            name="Conteos"
        ),
        row=1, col=1
    )

    # Matriz de confusión normalizada
    fig.add_trace(
        go.Heatmap(
            z=cm_normalized,
            x=class_names,
            y=class_names,
            colorscale='Reds',
            text=np.round(cm_normalized, 3),
            texttemplate="%{text}",
            textfont={"size": 12},
            showscale=True,
            colorbar=dict(x=1.0),
            name="Normalizada"
        ),
        row=1, col=2
    )

    # Actualizar layout
    fig.update_layout(
        title="Análisis de Matriz de Confusión",
        height=500,
        width=1000,
        showlegend=False
    )

    fig.update_xaxes(title_text="Predicción", row=1, col=1)
    fig.update_yaxes(title_text="Real", row=1, col=1)
    fig.update_xaxes(title_text="Predicción", row=1, col=2)
    fig.update_yaxes(title_text="Real", row=1, col=2)

    # Crear directorio si no existe
    os.makedirs(f'{RESULTS_PATH}/visualizations', exist_ok=True)

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/confusion_matrix.html')
    fig.show()

    print(f"📊 Matriz de confusión guardada en: {RESULTS_PATH}/visualizations/confusion_matrix.html")

def create_roc_curves(y_true_categorical, y_pred_proba, class_names):
    """Crear curvas ROC para cada clase"""

    fig = go.Figure()

    # Si es binario, tratar de forma especial
    if len(class_names) == 2:
        # Problema binario
        fpr, tpr, _ = roc_curve(y_true_categorical[:, 1], y_pred_proba[:, 1])
        roc_auc = auc(fpr, tpr)

        fig.add_trace(go.Scatter(
            x=fpr, y=tpr,
            mode='lines',
            name=f'{class_names[1]} vs {class_names[0]} (AUC = {roc_auc:.3f})',
            line=dict(width=3)
        ))
    else:
        # Problema multiclase - One-vs-Rest
        for i, class_name in enumerate(class_names):
            try:
                fpr, tpr, _ = roc_curve(y_true_categorical[:, i], y_pred_proba[:, i])
                roc_auc = auc(fpr, tpr)

                fig.add_trace(go.Scatter(
                    x=fpr, y=tpr,
                    mode='lines',
                    name=f'{class_name} (AUC = {roc_auc:.3f})',
                    line=dict(width=2)
                ))
            except Exception as e:
                print(f"⚠️ Error calculando ROC para {class_name}: {e}")
                continue

    # Línea diagonal de referencia
    fig.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Random Classifier',
        line=dict(dash='dash', color='gray', width=1)
    ))

    fig.update_layout(
        title='Curvas ROC por Clase',
        xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
        yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
        width=800,
        height=600,
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1])
    )

    # Crear directorio si no existe
    os.makedirs(f'{RESULTS_PATH}/visualizations', exist_ok=True)

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/roc_curves.html')
    fig.show()

    print(f"📈 Curvas ROC guardadas en: {RESULTS_PATH}/visualizations/roc_curves.html")

def create_metrics_dashboard(metrics, class_names):
    """Crear dashboard de métricas"""

    # Preparar datos para visualización
    metrics_data = []

    for class_name in class_names:
        metrics_data.append({
            'Clase': class_name,
            'Precisión': metrics[f'precision_{class_name}'],
            'Recall': metrics[f'recall_{class_name}'],
            'F1-Score': metrics[f'f1_{class_name}']
        })

    df_metrics = pd.DataFrame(metrics_data)

    # Crear gráfico de barras agrupadas
    fig = go.Figure()

    x = df_metrics['Clase']

    fig.add_trace(go.Bar(
        name='Precisión',
        x=x,
        y=df_metrics['Precisión'],
        marker_color='lightblue',
        text=np.round(df_metrics['Precisión'], 3),
        textposition='auto'
    ))

    fig.add_trace(go.Bar(
        name='Recall (Sensibilidad)',
        x=x,
        y=df_metrics['Recall'],
        marker_color='lightcoral',
        text=np.round(df_metrics['Recall'], 3),
        textposition='auto'
    ))

    fig.add_trace(go.Bar(
        name='F1-Score',
        x=x,
        y=df_metrics['F1-Score'],
        marker_color='lightgreen',
        text=np.round(df_metrics['F1-Score'], 3),
        textposition='auto'
    ))

    # Agregar líneas de referencia
    fig.add_hline(y=0.8, line_dash="dash", line_color="red",
                  annotation_text="Meta Clínica (0.8)")
    fig.add_hline(y=0.9, line_dash="dash", line_color="orange",
                  annotation_text="Excelencia Clínica (0.9)")

    fig.update_layout(
        title='Métricas de Rendimiento por Clase',
        xaxis_title='Clase',
        yaxis_title='Score',
        barmode='group',
        height=500,
        width=800,
        yaxis=dict(range=[0, 1.1]),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Crear directorio si no existe
    os.makedirs(f'{RESULTS_PATH}/visualizations', exist_ok=True)

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/metrics_dashboard.html')
    fig.show()

    print(f"📊 Dashboard de métricas guardado en: {RESULTS_PATH}/visualizations/metrics_dashboard.html")

def create_precision_recall_curves(y_true_categorical, y_pred_proba, class_names):
    """Crear curvas Precisión-Recall para cada clase"""

    fig = go.Figure()

    for i, class_name in enumerate(class_names):
        try:
            precision, recall, _ = precision_recall_curve(y_true_categorical[:, i], y_pred_proba[:, i])
            avg_precision = average_precision_score(y_true_categorical[:, i], y_pred_proba[:, i])

            fig.add_trace(go.Scatter(
                x=recall, y=precision,
                mode='lines',
                name=f'{class_name} (AP = {avg_precision:.3f})',
                line=dict(width=2)
            ))
        except Exception as e:
            print(f"⚠️ Error calculando Precisión-Recall para {class_name}: {e}")
            continue

    # Línea de referencia random
    baseline = np.sum(y_true_categorical, axis=0) / len(y_true_categorical)
    for i, class_name in enumerate(class_names):
        fig.add_hline(y=baseline[i], line_dash="dash", line_color="gray",
                      annotation_text=f"Baseline {class_name} ({baseline[i]:.3f})")

    fig.update_layout(
        title='Curvas Precisión-Recall por Clase',
        xaxis_title='Recall (Sensibilidad)',
        yaxis_title='Precisión (VPP)',
        width=800,
        height=600,
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1])
    )

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/precision_recall_curves.html')
    fig.show()

    print(f"📈 Curvas Precisión-Recall guardadas en: {RESULTS_PATH}/visualizations/precision_recall_curves.html")

def create_class_distribution_comparison(y_true, y_pred, class_names):
    """Comparar distribución de clases real vs predicha"""

    # Contar distribuciones
    true_counts = [np.sum(y_true == i) for i in range(len(class_names))]
    pred_counts = [np.sum(y_pred == i) for i in range(len(class_names))]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name='Real',
        x=class_names,
        y=true_counts,
        marker_color='skyblue',
        text=true_counts,
        textposition='auto'
    ))

    fig.add_trace(go.Bar(
        name='Predicho',
        x=class_names,
        y=pred_counts,
        marker_color='lightcoral',
        text=pred_counts,
        textposition='auto'
    ))

    fig.update_layout(
        title='Distribución de Clases: Real vs Predicha',
        xaxis_title='Clase',
        yaxis_title='Número de Muestras',
        barmode='group',
        height=500,
        width=700
    )

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/class_distribution.html')
    fig.show()

    print(f"📊 Distribución de clases guardada en: {RESULTS_PATH}/visualizations/class_distribution.html")

# Crear visualizaciones completas
print("\n📊 Creando visualizaciones completas de evaluación...")

try:
    # Matriz de confusión
    create_confusion_matrix_plot(
        evaluation_results['y_true'],
        evaluation_results['y_pred'],
        case_config['classes']
    )

    # Preparar y_true_categorical correctamente
    if use_test_generator:
        y_true_cat = tf.keras.utils.to_categorical(evaluation_results['y_true'], len(case_config['classes']))
    else:
        y_true_cat = y_test

    # Curvas ROC
    create_roc_curves(y_true_cat, evaluation_results['y_pred_proba'], case_config['classes'])

    # Dashboard de métricas
    create_metrics_dashboard(evaluation_results['metrics'], case_config['classes'])

    # Curvas Precisión-Recall
    create_precision_recall_curves(y_true_cat, evaluation_results['y_pred_proba'], case_config['classes'])

    # Distribución de clases
    create_class_distribution_comparison(evaluation_results['y_true'], evaluation_results['y_pred'], case_config['classes'])

    print("✅ Todas las visualizaciones creadas exitosamente!")

except Exception as e:
    print(f"❌ Error creando visualizaciones: {str(e)}")
    print("Continuando con el resto del notebook...")

#@title ## 🔍 Interpretabilidad con Grad-CAM - VERSION MEJORADA

class GradCAMVisualizer:
    """Clase mejorada para generar visualizaciones Grad-CAM"""

    def __init__(self, model, class_names):
        self.model = model
        self.class_names = class_names
        self.target_layers = self._find_target_layers()

    def _find_target_layers(self):
        """Encontrar capas objetivo para Grad-CAM"""
        target_layers = []

        for layer in reversed(self.model.layers):
            # Buscar capas convolucionales
            if 'conv' in layer.name.lower() or isinstance(layer, tf.keras.layers.Conv2D):
                target_layers.append(layer.name)
            # También considerar capas de activación después de convolucionales
            elif hasattr(layer, 'activation') and len(layer.output.shape) == 4:
                target_layers.append(layer.name)

        return target_layers[:3]  # Tomar las 3 mejores capas

    def generate_gradcam(self, image, class_index, layer_name=None):
        """Generar visualización Grad-CAM mejorada"""

        try:
            # Seleccionar capa objetivo
            if layer_name is None and self.target_layers:
                layer_name = self.target_layers[0]
            elif layer_name is None:
                print("⚠️ No se encontraron capas adecuadas para Grad-CAM")
                return None

            # Verificar que la capa existe
            try:
                target_layer = self.model.get_layer(layer_name)
            except ValueError:
                print(f"⚠️ Capa '{layer_name}' no encontrada")
                return None

            # Crear modelo Grad-CAM
            grad_model = keras.Model(
                inputs=self.model.inputs,
                outputs=[target_layer.output, self.model.output]
            )

            # Asegurar que la imagen tiene la forma correcta
            if len(image.shape) == 3:
                image_batch = np.expand_dims(image, axis=0)
            else:
                image_batch = image

            # Calcular gradientes
            with tf.GradientTape() as tape:
                conv_outputs, predictions = grad_model(image_batch)
                if len(predictions.shape) > 1:
                    loss = predictions[:, class_index]
                else:
                    loss = predictions[class_index]

            # Obtener gradientes
            grads = tape.gradient(loss, conv_outputs)

            if grads is None:
                print("⚠️ No se pudieron calcular gradientes")
                return None

            # Calcular pesos Grad-CAM
            pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

            # Generar heatmap
            conv_outputs = conv_outputs[0] if len(conv_outputs.shape) > 3 else conv_outputs
            heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
            heatmap = tf.squeeze(heatmap)

            # Normalizar heatmap
            heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)

            return heatmap.numpy()

        except Exception as e:
            print(f"❌ Error generando Grad-CAM: {str(e)}")
            return None

    def visualize_gradcam_samples(self, images, true_labels, predictions, file_paths=None, num_samples=6):
        """Visualizar Grad-CAM para muestras seleccionadas con mejor organización"""

        # Seleccionar muestras representativas
        samples_to_show = self._select_representative_samples(
            true_labels, predictions, num_samples
        )

        if not samples_to_show:
            print("⚠️ No se pudieron seleccionar muestras para Grad-CAM")
            return

        # Crear visualización mejorada
        fig, axes = plt.subplots(len(samples_to_show), 4, figsize=(20, 5 * len(samples_to_show)))
        if len(samples_to_show) == 1:
            axes = axes.reshape(1, -1)

        for idx, sample_idx in enumerate(samples_to_show):
            image = images[sample_idx]
            true_class_idx = np.argmax(true_labels[sample_idx]) if len(true_labels[sample_idx].shape) > 0 else true_labels[sample_idx]
            pred_class_idx = np.argmax(predictions[sample_idx])

            true_class = self.class_names[true_class_idx]
            pred_class = self.class_names[pred_class_idx]
            confidence = predictions[sample_idx, pred_class_idx]

            # Determinar si la predicción es correcta
            is_correct = true_class_idx == pred_class_idx
            color = 'green' if is_correct else 'red'

            # 1. Imagen original
            axes[idx, 0].imshow(image)
            axes[idx, 0].set_title(
                f'Original\n{true_class} → {pred_class}\nConf: {confidence:.3f}',
                color=color, fontsize=10
            )
            axes[idx, 0].axis('off')

            # 2. Grad-CAM para clase predicha
            heatmap_pred = self.generate_gradcam(image, pred_class_idx)
            if heatmap_pred is not None:
                # Redimensionar heatmap
                heatmap_resized = cv2.resize(heatmap_pred, (image.shape[1], image.shape[0]))

                # Superposición
                axes[idx, 1].imshow(image)
                im1 = axes[idx, 1].imshow(heatmap_resized, cmap='jet', alpha=0.4)
                axes[idx, 1].set_title(f'Grad-CAM: {pred_class}')

                # Agregar colorbar
                plt.colorbar(im1, ax=axes[idx, 1], fraction=0.046, pad=0.04)
            else:
                axes[idx, 1].imshow(image)
                axes[idx, 1].set_title('Grad-CAM no disponible')
            axes[idx, 1].axis('off')

            # 3. Grad-CAM para clase real (si es diferente)
            if true_class_idx != pred_class_idx:
                heatmap_true = self.generate_gradcam(image, true_class_idx)
                if heatmap_true is not None:
                    heatmap_resized = cv2.resize(heatmap_true, (image.shape[1], image.shape[0]))
                    axes[idx, 2].imshow(image)
                    im2 = axes[idx, 2].imshow(heatmap_resized, cmap='jet', alpha=0.4)
                    axes[idx, 2].set_title(f'Grad-CAM: {true_class}')
                    plt.colorbar(im2, ax=axes[idx, 2], fraction=0.046, pad=0.04)
                else:
                    axes[idx, 2].imshow(image)
                    axes[idx, 2].set_title('Grad-CAM no disponible')
            else:
                # Solo mostrar el heatmap puro
                if heatmap_pred is not None:
                    im2 = axes[idx, 2].imshow(heatmap_pred, cmap='jet')
                    axes[idx, 2].set_title('Mapa de Activación')
                    plt.colorbar(im2, ax=axes[idx, 2], fraction=0.046, pad=0.04)
                else:
                    axes[idx, 2].axis('off')
            axes[idx, 2].axis('off')

            # 4. Distribución de probabilidades
            probs = predictions[sample_idx]
            bars = axes[idx, 3].bar(range(len(self.class_names)), probs,
                                   color=['red' if i == pred_class_idx else 'lightblue'
                                         for i in range(len(self.class_names))])

            # Resaltar la clase real con borde
            if true_class_idx < len(bars):
                bars[true_class_idx].set_edgecolor('green')
                bars[true_class_idx].set_linewidth(3)

            axes[idx, 3].set_title('Probabilidades')
            axes[idx, 3].set_xticks(range(len(self.class_names)))
            axes[idx, 3].set_xticklabels(self.class_names, rotation=45, ha='right')
            axes[idx, 3].set_ylim([0, 1])

            # Agregar línea de confianza
            axes[idx, 3].axhline(y=0.5, color='gray', linestyle='--', alpha=0.7)

        plt.tight_layout()

        # Crear directorio si no existe
        os.makedirs(f'{RESULTS_PATH}/visualizations', exist_ok=True)

        plt.savefig(f'{RESULTS_PATH}/visualizations/gradcam_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()

        print(f"🔍 Análisis Grad-CAM guardado en: {RESULTS_PATH}/visualizations/gradcam_analysis.png")

    def _select_representative_samples(self, true_labels, predictions, num_samples):
        """Seleccionar muestras representativas para análisis"""

        true_classes = np.argmax(true_labels, axis=1) if len(true_labels.shape) > 1 else true_labels
        pred_classes = np.argmax(predictions, axis=1)

        samples = []

        # 1. Casos correctos con alta confianza
        correct_mask = true_classes == pred_classes
        if np.any(correct_mask):
            correct_indices = np.where(correct_mask)[0]
            max_probs = np.max(predictions, axis=1)
            high_conf_correct = correct_indices[np.argsort(max_probs[correct_indices])[-2:]]
            samples.extend(high_conf_correct)

        # 2. Casos incorrectos con alta confianza (errores confiados)
        incorrect_mask = true_classes != pred_classes
        if np.any(incorrect_mask):
            incorrect_indices = np.where(incorrect_mask)[0]
            max_probs = np.max(predictions, axis=1)
            high_conf_incorrect = incorrect_indices[np.argsort(max_probs[incorrect_indices])[-2:]]
            samples.extend(high_conf_incorrect)

        # 3. Casos con baja confianza
        max_probs = np.max(predictions, axis=1)
        low_conf_indices = np.where(max_probs < 0.7)[0]
        if len(low_conf_indices) > 0:
            samples.extend(np.random.choice(low_conf_indices,
                                          min(2, len(low_conf_indices)),
                                          replace=False))

        # Completar con muestras aleatorias si es necesario
        remaining = num_samples - len(samples)
        if remaining > 0:
            all_indices = set(range(len(predictions)))
            available = list(all_indices - set(samples))
            if available:
                samples.extend(np.random.choice(available,
                                              min(remaining, len(available)),
                                              replace=False))

        return samples[:num_samples]

# Crear visualizador Grad-CAM mejorado
print("\n🔍 Creando visualizaciones Grad-CAM para interpretabilidad...")

try:
    gradcam_visualizer = GradCAMVisualizer(best_model, case_config['classes'])

    # Generar visualizaciones Grad-CAM
    if not use_test_generator and X_test is not None:
        # Seleccionar subconjunto para Grad-CAM
        num_samples = min(8, len(X_test))
        gradcam_visualizer.visualize_gradcam_samples(
            X_test, y_test, evaluation_results['y_pred_proba'],
            test_file_paths, num_samples
        )
    else:
        print("⚠️ Grad-CAM requiere arrays numpy. Implementando versión alternativa...")

        # Alternativa para generadores
        if test_generator is not None:
            test_generator.reset()
            batch_images, batch_labels = test_generator.next()
            sample_predictions = best_model.predict(batch_images[:6])

            gradcam_visualizer.visualize_gradcam_samples(
                batch_images[:6], batch_labels[:6], sample_predictions
            )

except Exception as e:
    print(f"❌ Error en Grad-CAM: {str(e)}")
    print("Continuando sin visualizaciones Grad-CAM...")

#@title ## 📋 Análisis de Casos Difíciles - MEJORADO

def analyze_difficult_cases(y_true, y_pred, y_pred_proba, class_names, threshold=0.7):
    """Análisis completo de casos problemáticos"""

    print("\n🔍 ANÁLISIS COMPLETO DE CASOS DIFÍCILES")
    print("=" * 60)

    # Métricas básicas
    max_proba = np.max(y_pred_proba, axis=1)
    low_confidence_cases = np.where(max_proba < threshold)[0]
    incorrect_cases = np.where(y_true != y_pred)[0]

    total_samples = len(y_true)
    low_conf_pct = (len(low_confidence_cases) / total_samples) * 100
    error_pct = (len(incorrect_cases) / total_samples) * 100

    print(f"📊 RESUMEN GENERAL:")
    print(f"  Total de muestras: {total_samples}")
    print(f"  Casos con confianza < {threshold}: {len(low_confidence_cases)} ({low_conf_pct:.1f}%)")
    print(f"  Predicciones incorrectas: {len(incorrect_cases)} ({error_pct:.1f}%)")

    # Casos críticos (incorrectos con alta confianza)
    critical_cases = np.intersect1d(
        incorrect_cases,
        np.where(max_proba >= 0.8)[0]
    )
    print(f"  🚨 Casos críticos (error + alta confianza): {len(critical_cases)}")

    # Análisis por clase
    print(f"\n📈 ANÁLISIS DETALLADO POR CLASE:")
    class_analysis = {}

    for i, class_name in enumerate(class_names):
        class_mask = y_true == i
        class_total = np.sum(class_mask)

        if class_total == 0:
            continue

        class_correct = np.sum((y_true == y_pred) & class_mask)
        class_accuracy = class_correct / class_total

        # Casos problemáticos de esta clase
        class_low_conf = np.sum(class_mask & (max_proba < threshold))
        class_false_neg = np.sum(class_mask & (y_pred != i))  # Falsos negativos
        class_false_pos = np.sum((y_pred == i) & (y_true != i))  # Falsos positivos

        # Confianza promedio para esta clase
        class_probs = y_pred_proba[class_mask, i]
        avg_confidence = np.mean(class_probs) if len(class_probs) > 0 else 0

        class_analysis[class_name] = {
            'total': class_total,
            'correct': class_correct,
            'accuracy': class_accuracy,
            'low_confidence': class_low_conf,
            'false_negatives': class_false_neg,
            'false_positives': class_false_pos,
            'avg_confidence': avg_confidence
        }

        print(f"\n  🏷️ {class_name.upper()}:")
        print(f"    Total: {class_total} | Correctos: {class_correct} | Precisión: {class_accuracy:.3f}")
        print(f"    Baja confianza: {class_low_conf} | Falsos negativos: {class_false_neg}")
        print(f"    Falsos positivos: {class_false_pos} | Confianza promedio: {avg_confidence:.3f}")

        # Alertas específicas
        if class_accuracy < 0.8:
            print(f"    ⚠️ ALERTA: Precisión baja para {class_name}")
        if class_false_neg > class_total * 0.15:  # >15% falsos negativos
            print(f"    🚨 CRÍTICO: Alto índice de falsos negativos para {class_name}")

    # Análisis de patrones de error
    print(f"\n🔄 PATRONES DE ERRORES MÁS FRECUENTES:")
    if len(incorrect_cases) > 0:
        error_patterns = {}
        for idx in incorrect_cases:
            true_class = class_names[y_true[idx]]
            pred_class = class_names[y_pred[idx]]
            pattern = f"{true_class} → {pred_class}"
            error_patterns[pattern] = error_patterns.get(pattern, 0) + 1

        # Ordenar por frecuencia
        sorted_errors = sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)
        for pattern, count in sorted_errors[:5]:  # Top 5 errores
            percentage = (count / len(incorrect_cases)) * 100
            print(f"  {pattern}: {count} casos ({percentage:.1f}% de errores)")

    # Análisis de confianza por rangos
    print(f"\n📊 DISTRIBUCIÓN DE CONFIANZA:")
    confidence_ranges = [
        (0.0, 0.5, "Muy baja"),
        (0.5, 0.7, "Baja"),
        (0.7, 0.8, "Media"),
        (0.8, 0.9, "Alta"),
        (0.9, 1.0, "Muy alta")
    ]

    for min_conf, max_conf, label in confidence_ranges:
        range_mask = (max_proba >= min_conf) & (max_proba < max_conf)
        range_count = np.sum(range_mask)
        range_accuracy = np.mean(y_true[range_mask] == y_pred[range_mask]) if range_count > 0 else 0
        range_pct = (range_count / total_samples) * 100

        print(f"  {label} ({min_conf}-{max_conf}): {range_count} casos ({range_pct:.1f}%) - Precisión: {range_accuracy:.3f}")

    # Recomendaciones específicas
    print(f"\n💡 RECOMENDACIONES BASADAS EN EL ANÁLISIS:")
    recommendations = []

    if low_conf_pct > 20:
        recommendations.append("Alto porcentaje de casos con baja confianza - considerar ensemble de modelos")

    if len(critical_cases) > total_samples * 0.05:  # >5% casos críticos
        recommendations.append("Casos críticos detectados - revisar arquitectura del modelo")

    # Recomendaciones por clase
    for class_name, analysis in class_analysis.items():
        if analysis['false_negatives'] > analysis['total'] * 0.15:
            recommendations.append(f"Mejorar detección de clase '{class_name}' - muchos falsos negativos")
        if analysis['avg_confidence'] < 0.6:
            recommendations.append(f"Baja confianza en clase '{class_name}' - revisar datos de entrenamiento")

    if recommendations:
        for i, rec in enumerate(recommendations, 1):
            print(f"  {i}. {rec}")
    else:
        print("  ✅ No se detectaron problemas críticos en el modelo")

    return {
        'low_confidence_cases': low_confidence_cases,
        'incorrect_cases': incorrect_cases,
        'critical_cases': critical_cases,
        'error_patterns': error_patterns if len(incorrect_cases) > 0 else {},
        'class_analysis': class_analysis,
        'recommendations': recommendations
    }

def create_confidence_distribution_plot(y_pred_proba, y_true, y_pred, class_names):
    """Crear análisis completo de distribución de confianza"""

    max_proba = np.max(y_pred_proba, axis=1)
    correct_mask = y_true == y_pred

    # Crear figura con múltiples subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Distribución General de Confianza',
            'Confianza por Clase Predicha',
            'Confianza vs Precisión',
            'Casos Problemáticos'
        ],
        specs=[[{'type': 'histogram'}, {'type': 'box'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )

    # 1. Distribución general
    fig.add_trace(go.Histogram(
        x=max_proba[correct_mask],
        name='Correctas',
        opacity=0.7,
        nbinsx=20,
        marker_color='green',
        legendgroup='correct'
    ), row=1, col=1)

    fig.add_trace(go.Histogram(
        x=max_proba[~correct_mask],
        name='Incorrectas',
        opacity=0.7,
        nbinsx=20,
        marker_color='red',
        legendgroup='incorrect'
    ), row=1, col=1)

    # Líneas de referencia
    fig.add_vline(x=0.5, line_dash="dash", line_color="gray", row=1, col=1)
    fig.add_vline(x=0.7, line_dash="dash", line_color="blue", row=1, col=1)
    fig.add_vline(x=0.9, line_dash="dash", line_color="orange", row=1, col=1)

    # 2. Box plots por clase predicha
    pred_classes = np.argmax(y_pred_proba, axis=1)
    for i, class_name in enumerate(class_names):
        class_mask = pred_classes == i
        if np.any(class_mask):
            fig.add_trace(go.Box(
                y=max_proba[class_mask],
                name=class_name,
                boxpoints='outliers'
            ), row=1, col=2)

    # 3. Confianza vs precisión por bins
    confidence_bins = np.linspace(0, 1, 11)
    bin_centers = (confidence_bins[:-1] + confidence_bins[1:]) / 2
    bin_accuracies = []
    bin_counts = []

    for i in range(len(confidence_bins)-1):
        bin_mask = (max_proba >= confidence_bins[i]) & (max_proba < confidence_bins[i+1])
        if np.any(bin_mask):
            bin_accuracy = np.mean(correct_mask[bin_mask])
            bin_count = np.sum(bin_mask)
        else:
            bin_accuracy = 0
            bin_count = 0
        bin_accuracies.append(bin_accuracy)
        bin_counts.append(bin_count)

    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=bin_accuracies,
        mode='lines+markers',
        name='Precisión por Confianza',
        line=dict(width=3),
        marker=dict(size=8)
    ), row=2, col=1)

    # Línea de calibración perfecta
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Calibración Perfecta',
        line=dict(dash='dash', color='gray')
    ), row=2, col=1)

    # 4. Análisis de casos problemáticos
    problem_categories = ['Muy Baja (<0.5)', 'Baja (0.5-0.7)', 'Errores Alta Conf (>0.8)']
    problem_counts = [
        np.sum(max_proba < 0.5),
        np.sum((max_proba >= 0.5) & (max_proba < 0.7)),
        np.sum((max_proba >= 0.8) & ~correct_mask)
    ]

    colors = ['red', 'orange', 'darkred']
    fig.add_trace(go.Bar(
        x=problem_categories,
        y=problem_counts,
        marker_color=colors,
        text=problem_counts,
        textposition='auto'
    ), row=2, col=2)

    # Actualizar layout
    fig.update_layout(
        title='Análisis Completo de Confianza del Modelo',
        height=800,
        width=1200,
        showlegend=True
    )

    fig.update_xaxes(title_text="Confianza Máxima", row=1, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=1, col=1)
    fig.update_yaxes(title_text="Confianza", row=1, col=2)
    fig.update_xaxes(title_text="Confianza", row=2, col=1)
    fig.update_yaxes(title_text="Precisión", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Problema", row=2, col=2)
    fig.update_yaxes(title_text="Número de Casos", row=2, col=2)

    # Crear directorio si no existe
    os.makedirs(f'{RESULTS_PATH}/visualizations', exist_ok=True)

    # Guardar y mostrar
    fig.write_html(f'{RESULTS_PATH}/visualizations/confidence_analysis.html')
    fig.show()

    print(f"📊 Análisis de confianza guardado en: {RESULTS_PATH}/visualizations/confidence_analysis.html")

def create_error_analysis_plot(y_true, y_pred, y_pred_proba, class_names):
    """Crear visualización detallada del análisis de errores"""

    # Calcular matriz de confusión
    cm = confusion_matrix(y_true, y_pred)

    # Crear heatmap de errores (solo errores, diagonal en 0)
    error_matrix = cm.copy()
    np.fill_diagonal(error_matrix, 0)

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Matriz de Errores (sin diagonal)',
            'Errores por Confianza',
            'Top Confusiones',
            'Distribución de Errores por Clase'
        ],
        specs=[[{'type': 'heatmap'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )

    # 1. Matriz de errores
    fig.add_trace(go.Heatmap(
        z=error_matrix,
        x=class_names,
        y=class_names,
        colorscale='Reds',
        text=error_matrix,
        texttemplate="%{text}",
        showscale=True
    ), row=1, col=1)

    # 2. Errores vs confianza
    incorrect_mask = y_true != y_pred
    max_proba = np.max(y_pred_proba, axis=1)

    fig.add_trace(go.Scatter(
        x=max_proba[incorrect_mask],
        y=np.ones(np.sum(incorrect_mask)),
        mode='markers',
        name='Errores',
        marker=dict(color='red', size=8, opacity=0.6),
        yaxis='y2'
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=max_proba[~incorrect_mask],
        y=np.zeros(np.sum(~incorrect_mask)),
        mode='markers',
        name='Correctas',
        marker=dict(color='green', size=6, opacity=0.4),
        yaxis='y2'
    ), row=1, col=2)

    # 3. Top confusiones
    error_pairs = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            if i != j and error_matrix[i,j] > 0:
                error_pairs.append((f"{class_names[i]}→{class_names[j]}", error_matrix[i,j]))

    # Ordenar y tomar top 5
    error_pairs.sort(key=lambda x: x[1], reverse=True)
    top_errors = error_pairs[:min(5, len(error_pairs))]

    if top_errors:
        error_labels, error_counts = zip(*top_errors)
        fig.add_trace(go.Bar(
            x=list(error_labels),
            y=list(error_counts),
            marker_color='red',
            text=list(error_counts),
            textposition='auto'
        ), row=2, col=1)

    # 4. Errores por clase (falsos negativos)
    false_negatives = []
    for i, class_name in enumerate(class_names):
        fn_count = np.sum((y_true == i) & (y_pred != i))
        false_negatives.append(fn_count)

    fig.add_trace(go.Bar(
        x=class_names,
        y=false_negatives,
        marker_color='orange',
        text=false_negatives,
        textposition='auto',
        name='Falsos Negativos'
    ), row=2, col=2)

    # Actualizar layout
    fig.update_layout(
        title='Análisis Detallado de Errores del Modelo',
        height=800,
        width=1200
    )

    fig.update_xaxes(title_text="Predicción", row=1, col=1)
    fig.update_yaxes(title_text="Real", row=1, col=1)
    fig.update_xaxes(title_text="Confianza", row=1, col=2)
    fig.update_yaxes(title_text="Error (1) / Correcto (0)", row=1, col=2)
    fig.update_xaxes(title_text="Confusión", row=2, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=2, col=1)
    fig.update_xaxes(title_text="Clase", row=2, col=2)
    fig.update_yaxes(title_text="Falsos Negativos", row=2, col=2)

    # Guardar
    fig.write_html(f'{RESULTS_PATH}/visualizations/error_analysis.html')
    fig.show()

    print(f"🔍 Análisis de errores guardado en: {RESULTS_PATH}/visualizations/error_analysis.html")

# Realizar análisis completo de casos difíciles
print("🔍 Iniciando análisis completo de casos difíciles...")

difficult_cases_analysis = analyze_difficult_cases(
    evaluation_results['y_true'],
    evaluation_results['y_pred'],
    evaluation_results['y_pred_proba'],
    case_config['classes'],
    threshold=0.7
)

# Crear visualizaciones de confianza y errores
create_confidence_distribution_plot(
    evaluation_results['y_pred_proba'],
    evaluation_results['y_true'],
    evaluation_results['y_pred'],
    case_config['classes']
)

create_error_analysis_plot(
    evaluation_results['y_true'],
    evaluation_results['y_pred'],
    evaluation_results['y_pred_proba'],
    case_config['classes']
)

print("✅ Análisis de casos difíciles completado!")

#@title ## 📄 Generación de Reporte Clínico Detallado - VERSION COMPLETA

class ClinicalReportGenerator:
    """Generador completo de reportes clínicos para diagnóstico médico"""

    def __init__(self, case_type, class_names):
        self.case_type = case_type
        self.class_names = class_names
        self.report_date = datetime.now()

    def generate_clinical_report(self, evaluation_results, model_metadata, difficult_cases):
        """Generar reporte clínico completo y detallado"""

        metrics = evaluation_results['metrics']

        report = {
            'report_metadata': {
                'generation_date': self.report_date.isoformat(),
                'report_version': '1.0',
                'case_type': self.case_type,
                'model_name': model_metadata.get('model_name', 'Unknown'),
                'evaluation_samples': len(evaluation_results['y_true']),
                'classes_evaluated': self.class_names,
                'evaluation_framework': 'DataLabPro AI v1.0'
            },

            'performance_summary': {
                'overall_accuracy': metrics['accuracy'],
                'macro_f1_score': metrics['f1_macro'],
                'macro_precision': metrics['precision_macro'],
                'macro_recall': metrics['recall_macro'],
                'performance_grade': self._calculate_performance_grade(metrics['accuracy'])
            },

            'clinical_metrics': self._extract_clinical_metrics(metrics),

            'class_performance': self._analyze_class_performance(metrics),

            'reliability_analysis': {
                'total_predictions': len(evaluation_results['y_true']),
                'low_confidence_cases': len(difficult_cases['low_confidence_cases']),
                'low_confidence_percentage': len(difficult_cases['low_confidence_cases']) / len(evaluation_results['y_true']) * 100,
                'incorrect_predictions': len(difficult_cases['incorrect_cases']),
                'error_rate': len(difficult_cases['incorrect_cases']) / len(evaluation_results['y_true']) * 100,
                'critical_cases': len(difficult_cases.get('critical_cases', [])),
                'error_patterns': difficult_cases.get('error_patterns', {}),
                'reliability_score': self._calculate_reliability_score(difficult_cases, len(evaluation_results['y_true']))
            },

            'clinical_assessment': self._generate_clinical_assessment(metrics, difficult_cases),

            'recommendations': self._generate_recommendations(metrics, difficult_cases),

            'limitations_and_warnings': self._generate_limitations(),

            'regulatory_compliance': self._generate_regulatory_notes(),

            'next_steps': self._generate_next_steps(metrics, difficult_cases)
        }

        return report

    def _calculate_performance_grade(self, accuracy):
        """Asignar grado de rendimiento clínico"""
        if accuracy >= 0.95:
            return "Excelente (A+)"
        elif accuracy >= 0.90:
            return "Muy Bueno (A)"
        elif accuracy >= 0.85:
            return "Bueno (B+)"
        elif accuracy >= 0.80:
            return "Aceptable (B)"
        elif accuracy >= 0.75:
            return "Marginal (C)"
        else:
            return "Insuficiente (D)"

    def _extract_clinical_metrics(self, metrics):
        """Extraer métricas específicamente clínicas"""
        clinical_metrics = {}

        if self.case_type == 'breast_cancer':
            # Métricas específicas para cáncer de mama
            if 'sensitivity_malignant' in metrics:
                clinical_metrics['sensitivity'] = {
                    'value': metrics['sensitivity_malignant'],
                    'interpretation': self._interpret_sensitivity(metrics['sensitivity_malignant']),
                    'clinical_standard': 0.90,
                    'meets_standard': metrics['sensitivity_malignant'] >= 0.90
                }

            if 'specificity_malignant' in metrics:
                clinical_metrics['specificity'] = {
                    'value': metrics['specificity_malignant'],
                    'interpretation': self._interpret_specificity(metrics['specificity_malignant']),
                    'clinical_standard': 0.85,
                    'meets_standard': metrics['specificity_malignant'] >= 0.85
                }

            if 'ppv_malignant' in metrics:
                clinical_metrics['positive_predictive_value'] = {
                    'value': metrics['ppv_malignant'],
                    'interpretation': self._interpret_ppv(metrics['ppv_malignant'])
                }

            if 'npv_malignant' in metrics:
                clinical_metrics['negative_predictive_value'] = {
                    'value': metrics['npv_malignant'],
                    'interpretation': self._interpret_npv(metrics['npv_malignant'])
                }

        elif self.case_type == 'brain_tumor':
            # Métricas específicas para tumores cerebrales
            # Se pueden agregar métricas específicas aquí
            pass

        # Métricas generales aplicables a cualquier caso
        clinical_metrics['diagnostic_accuracy'] = {
            'value': metrics['accuracy'],
            'interpretation': self._interpret_accuracy(metrics['accuracy']),
            'confidence_interval': '95% CI: [{:.3f}, {:.3f}]'.format(
                max(0, metrics['accuracy'] - 0.05),
                min(1, metrics['accuracy'] + 0.05)
            )
        }

        return clinical_metrics

    def _interpret_sensitivity(self, sensitivity):
        """Interpretar valor de sensibilidad"""
        if sensitivity >= 0.95:
            return "Excelente capacidad para detectar casos positivos"
        elif sensitivity >= 0.90:
            return "Muy buena capacidad de detección"
        elif sensitivity >= 0.85:
            return "Capacidad de detección aceptable"
        elif sensitivity >= 0.80:
            return "Capacidad de detección marginal"
        else:
            return "Capacidad de detección insuficiente - riesgo de falsos negativos"

    def _interpret_specificity(self, specificity):
        """Interpretar valor de especificidad"""
        if specificity >= 0.95:
            return "Excelente capacidad para evitar falsos positivos"
        elif specificity >= 0.90:
            return "Muy buena especificidad"
        elif specificity >= 0.85:
            return "Especificidad aceptable"
        elif specificity >= 0.80:
            return "Especificidad marginal"
        else:
            return "Especificidad insuficiente - alto riesgo de falsos positivos"

    def _interpret_ppv(self, ppv):
        """Interpretar Valor Predictivo Positivo"""
        if ppv >= 0.90:
            return "Muy alta confiabilidad en diagnósticos positivos"
        elif ppv >= 0.80:
            return "Alta confiabilidad en diagnósticos positivos"
        elif ppv >= 0.70:
            return "Confiabilidad moderada en diagnósticos positivos"
        else:
            return "Baja confiabilidad - requiere confirmación adicional"

    def _interpret_npv(self, npv):
        """Interpretar Valor Predictivo Negativo"""
        if npv >= 0.95:
            return "Muy alta confiabilidad en diagnósticos negativos"
        elif npv >= 0.90:
            return "Alta confiabilidad en diagnósticos negativos"
        elif npv >= 0.85:
            return "Confiabilidad moderada en diagnósticos negativos"
        else:
            return "Baja confiabilidad en diagnósticos negativos"

    def _interpret_accuracy(self, accuracy):
        """Interpretar precisión diagnóstica"""
        if accuracy >= 0.95:
            return "Precisión diagnóstica excepcional"
        elif accuracy >= 0.90:
            return "Precisión diagnóstica muy alta"
        elif accuracy >= 0.85:
            return "Precisión diagnóstica alta"
        elif accuracy >= 0.80:
            return "Precisión diagnóstica aceptable para uso clínico"
        elif accuracy >= 0.75:
            return "Precisión diagnóstica marginal - requiere supervisión"
        else:
            return "Precisión diagnóstica insuficiente para uso clínico"

    def _analyze_class_performance(self, metrics):
        """Analizar rendimiento por clase con interpretación clínica"""
        class_analysis = {}

        for class_name in self.class_names:
            precision = metrics[f'precision_{class_name}']
            recall = metrics[f'recall_{class_name}']
            f1 = metrics[f'f1_{class_name}']

            class_analysis[class_name] = {
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'clinical_interpretation': self._interpret_class_performance(
                    class_name, precision, recall, f1
                ),
                'risk_assessment': self._assess_clinical_risk(class_name, precision, recall)
            }

        return class_analysis

    def _interpret_class_performance(self, class_name, precision, recall, f1):
        """Interpretar rendimiento de una clase específica"""
        interpretation = []

        if recall < 0.85:
            interpretation.append(f"Riesgo de falsos negativos en {class_name}")
        if precision < 0.80:
            interpretation.append(f"Riesgo de falsos positivos en {class_name}")
        if f1 >= 0.90:
            interpretation.append(f"Excelente balance precision-recall para {class_name}")
        elif f1 >= 0.80:
            interpretation.append(f"Buen rendimiento general para {class_name}")
        else:
            interpretation.append(f"Rendimiento subóptimo para {class_name}")

        return interpretation

    def _assess_clinical_risk(self, class_name, precision, recall):
        """Evaluar riesgo clínico por clase"""
        risk_level = "Bajo"
        risk_factors = []

        # Evaluar riesgo específico por tipo de clase
        if class_name.lower() in ['malignant', 'cancer', 'tumor', 'positive']:
            # Clase crítica - falsos negativos muy peligrosos
            if recall < 0.90:
                risk_level = "Alto"
                risk_factors.append("Falsos negativos en clase crítica")
            elif recall < 0.95:
                risk_level = "Medio"
                risk_factors.append("Sensibilidad por debajo del ideal")

        if precision < 0.70:
            if risk_level == "Bajo":
                risk_level = "Medio"
            risk_factors.append("Alta tasa de falsos positivos")

        return {
            'level': risk_level,
            'factors': risk_factors
        }

    def _calculate_reliability_score(self, difficult_cases, total_samples):
        """Calcular puntuación de confiabilidad del modelo"""

        low_conf_rate = len(difficult_cases['low_confidence_cases']) / total_samples
        error_rate = len(difficult_cases['incorrect_cases']) / total_samples
        critical_rate = len(difficult_cases.get('critical_cases', [])) / total_samples

        # Penalizar casos problemáticos
        reliability = 1.0 - (error_rate * 1.0 + low_conf_rate * 0.5 + critical_rate * 1.5)
        reliability = max(0, min(1, reliability))  # Mantener en [0,1]

        return {
            'score': reliability,
            'grade': 'A' if reliability >= 0.9 else 'B' if reliability >= 0.8 else 'C' if reliability >= 0.7 else 'D'
        }

    def _generate_clinical_assessment(self, metrics, difficult_cases):
        """Generar evaluación clínica integral"""

        assessment = {
            'overall_assessment': "",
            'strengths': [],
            'weaknesses': [],
            'clinical_readiness': "",
            'risk_factors': []
        }

        accuracy = metrics['accuracy']

        # Evaluación general
        if accuracy >= 0.90:
            assessment['overall_assessment'] = "El modelo demuestra un rendimiento excelente con potencial para uso clínico bajo supervisión apropiada."
        elif accuracy >= 0.85:
            assessment['overall_assessment'] = "El modelo muestra un rendimiento bueno, adecuado para uso clínico con monitoreo continuo."
        elif accuracy >= 0.80:
            assessment['overall_assessment'] = "El modelo presenta rendimiento aceptable pero requiere mejoras antes del despliegue clínico."
        else:
            assessment['overall_assessment'] = "El modelo requiere mejoras significativas antes de considerar uso clínico."

        # Identificar fortalezas
        if metrics['f1_macro'] >= 0.85:
            assessment['strengths'].append("Excelente balance entre precisión y recall")

        if self.case_type == 'breast_cancer' and 'sensitivity_malignant' in metrics:
            if metrics['sensitivity_malignant'] >= 0.90:
                assessment['strengths'].append("Alta sensibilidad para detección de malignidad")
            if metrics['specificity_malignant'] >= 0.85:
                assessment['strengths'].append("Buena especificidad, minimiza falsos positivos")

        low_conf_rate = len(difficult_cases['low_confidence_cases']) / len(difficult_cases.get('incorrect_cases', [1]))
        if low_conf_rate < 0.15:
            assessment['strengths'].append("Baja tasa de casos con confianza dudosa")

        # Identificar debilidades
        if accuracy < 0.85:
            assessment['weaknesses'].append("Precisión general por debajo del estándar clínico recomendado")

        if len(difficult_cases.get('critical_cases', [])) > 0:
            assessment['weaknesses'].append("Presencia de errores con alta confianza (casos críticos)")

        error_rate = len(difficult_cases['incorrect_cases']) / metrics.get('total_samples', 100)
        if error_rate > 0.15:
            assessment['weaknesses'].append("Tasa de error elevada")

        # Evaluación de preparación clínica
        critical_issues = len([w for w in assessment['weaknesses'] if 'crítico' in w.lower() or 'error' in w.lower()])

        if accuracy >= 0.90 and critical_issues == 0:
            assessment['clinical_readiness'] = "Listo para piloto clínico con supervisión médica"
        elif accuracy >= 0.85 and critical_issues <= 1:
            assessment['clinical_readiness'] = "Requiere validación adicional antes del uso clínico"
        else:
            assessment['clinical_readiness'] = "No listo para uso clínico - requiere mejoras significativas"

        # Factores de riesgo
        if self.case_type == 'breast_cancer':
            if metrics.get('sensitivity_malignant', 1.0) < 0.85:
                assessment['risk_factors'].append("Riesgo elevado de falsos negativos en casos de malignidad")
            if metrics.get('specificity_malignant', 1.0) < 0.80:
                assessment['risk_factors'].append("Riesgo de sobrediagnóstico por falsos positivos")

        return assessment

    def _generate_recommendations(self, metrics, difficult_cases):
        """Generar recomendaciones específicas y accionables"""

        recommendations = {
            'immediate_actions': [],
            'model_improvements': [],
            'clinical_implementation': [],
            'monitoring_requirements': [],
            'validation_steps': []
        }

        accuracy = metrics['accuracy']
        error_rate = len(difficult_cases['incorrect_cases']) / metrics.get('total_samples', 1)
        low_conf_rate = len(difficult_cases['low_confidence_cases']) / metrics.get('total_samples', 1)

        # Acciones inmediatas
        if len(difficult_cases.get('critical_cases', [])) > 0:
            recommendations['immediate_actions'].append(
                "Revisar casos críticos (errores con alta confianza) para identificar patrones"
            )

        if low_conf_rate > 0.20:
            recommendations['immediate_actions'].append(
                "Implementar sistema de revisión manual para casos de baja confianza"
            )

        # Mejoras del modelo
        if accuracy < 0.85:
            recommendations['model_improvements'].extend([
                "Aumentar diversidad del dataset de entrenamiento",
                "Considerar arquitecturas más complejas o ensemble de modelos",
                "Revisar y mejorar calidad de las etiquetas"
            ])

        if self.case_type == 'breast_cancer':
            if metrics.get('sensitivity_malignant', 1.0) < 0.90:
                recommendations['model_improvements'].append(
                    "Aplicar técnicas de balanceo para mejorar detección de malignidad"
                )

        # Implementación clínica
        recommendations['clinical_implementation'].extend([
            "Establecer protocolo de second opinion para casos dudosos",
            "Capacitar personal médico en interpretación de resultados del modelo",
            "Definir workflow de escalación para casos problemáticos"
        ])

        if accuracy >= 0.85:
            recommendations['clinical_implementation'].append(
                "Iniciar piloto controlado con supervisión médica estrecha"
            )

        # Requerimientos de monitoreo
        recommendations['monitoring_requirements'].extend([
            "Implementar tracking de deriva de datos en tiempo real",
            "Monitorear distribución de confianza de predicciones",
            "Establecer alertas para cambios en patrones de error",
            "Revisar performance mensualmente con nuevos casos"
        ])

        # Pasos de validación
        recommendations['validation_steps'].extend([
            "Validar con datos de diferentes centros médicos",
            "Realizar análisis de sesgo en diferentes poblaciones",
            "Comparar performance con radiólogos expertos",
            "Documentar casos límite y decisiones clínicas"
        ])

        return recommendations

    def _generate_limitations(self):
        """Generar limitaciones y consideraciones importantes"""

        limitations = [
            "Este modelo es una herramienta de apoyo diagnóstico y NO reemplaza el juicio clínico profesional",
            "La performance puede variar con diferentes equipos de imagen o protocolos de adquisición",
            "Requiere validación continua con datos de la población local",
            "No validado para casos con comorbilidades complejas o presentaciones atípicas"
        ]

        if self.case_type == 'breast_cancer':
            limitations.extend([
                "Validado específicamente para mamografías digitales estándar",
                "Performance puede diferir en implantes mamarios o tejido muy denso",
                "No incluye correlación con historial familiar o factores de riesgo genético"
            ])
        elif self.case_type == 'brain_tumor':
            limitations.extend([
                "Validado para secuencias específicas de resonancia magnética",
                "No diferencia subtipos histológicos específicos",
                "Requiere correlación con síntomas clínicos"
            ])

        return limitations

    def _generate_regulatory_notes(self):
        """Generar notas sobre cumplimiento regulatorio"""

        return {
            'fda_considerations': [
                "Modelo requiere validación clínica antes de aprobación FDA",
                "Documentación completa de dataset y metodología requerida",
                "Plan de post-market surveillance necesario"
            ],
            'hipaa_compliance': [
                "Datos utilizados fueron anonimizados según estándares HIPAA",
                "Modelo no almacena información identificable de pacientes",
                "Implementar controles de acceso apropiados en producción"
            ],
            'international_standards': [
                "Considera lineamientos ISO 14155 para investigación clínica",
                "Evaluar conformidad con regulaciones locales antes del despliegue",
                "Documentar según estándares IEC 62304 para software médico"
            ]
        }

    def _generate_next_steps(self, metrics, difficult_cases):
        """Generar próximos pasos específicos"""

        next_steps = []

        # Basado en performance
        if metrics['accuracy'] >= 0.90:
            next_steps.extend([
                "Preparar documentación para validación clínica",
                "Diseñar estudio prospectivo con radiólogos",
                "Desarrollar interfaz clínica user-friendly"
            ])
        elif metrics['accuracy'] >= 0.85:
            next_steps.extend([
                "Mejorar modelo con técnicas avanzadas",
                "Ampliar dataset con casos difíciles",
                "Realizar validación cruzada exhaustiva"
            ])
        else:
            next_steps.extend([
                "Revisar arquitectura del modelo completamente",
                "Auditar calidad del dataset",
                "Considerar enfoque completamente nuevo"
            ])

        # Basado en casos difíciles
        if len(difficult_cases.get('critical_cases', [])) > 0:
            next_steps.append("Análisis profundo de casos críticos con expertos médicos")

        if len(difficult_cases['low_confidence_cases']) > len(difficult_cases['incorrect_cases']) * 0.5:
            next_steps.append("Implementar técnicas de calibración de confianza")

        return next_steps

    def save_report_json(self, report, filename='clinical_evaluation_report.json'):
        """Guardar reporte como JSON estructurado"""

        # Crear directorio si no existe
        os.makedirs(f'{RESULTS_PATH}/reports', exist_ok=True)

        filepath = f'{RESULTS_PATH}/reports/{filename}'
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False, default=str)

        return filepath

    def save_report_html(self, report, filename='clinical_evaluation_report.html'):
        """Guardar reporte como HTML legible"""

        html_content = self._generate_html_report(report)
        filepath = f'{RESULTS_PATH}/reports/{filename}'

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(html_content)

        return filepath

    def _generate_html_report(self, report):
        """Generar reporte HTML formateado"""

        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Reporte Clínico - {report['report_metadata']['case_type']}</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }}
                .header {{ background-color: #f4f4f4; padding: 20px; border-radius: 5px; }}
                .section {{ margin: 20px 0; padding: 15px; border-left: 4px solid #007acc; }}
                .metric {{ background-color: #f9f9f9; padding: 10px; margin: 5px 0; }}
                .warning {{ background-color: #fff3cd; padding: 10px; border: 1px solid #ffeaa7; }}
                .success {{ background-color: #d4edda; padding: 10px; border: 1px solid #c3e6cb; }}
                .error {{ background-color: #f8d7da; padding: 10px; border: 1px solid #f5c6cb; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Reporte de Evaluación Clínica</h1>
                <h2>{report['report_metadata']['case_type'].replace('_', ' ').title()}</h2>
                <p><strong>Fecha:</strong> {report['report_metadata']['generation_date'][:10]}</p>
                <p><strong>Modelo:</strong> {report['report_metadata']['model_name']}</p>
                <p><strong>Muestras Evaluadas:</strong> {report['report_metadata']['evaluation_samples']}</p>
            </div>
        """

        # Resumen de performance
        html += f"""
            <div class="section">
                <h3>📊 Resumen de Rendimiento</h3>
                <div class="metric">
                    <strong>Precisión General:</strong> {report['performance_summary']['overall_accuracy']:.3f}
                    <span style="margin-left: 20px;">Grado: {report['performance_summary']['performance_grade']}</span>
                </div>
                <div class="metric"><strong>F1-Score Macro:</strong> {report['performance_summary']['macro_f1_score']:.3f}</div>
                <div class="metric"><strong>Precisión Macro:</strong> {report['performance_summary']['macro_precision']:.3f}</div>
                <div class="metric"><strong>Recall Macro:</strong> {report['performance_summary']['macro_recall']:.3f}</div>
            </div>
        """

        # Métricas clínicas
        if report['clinical_metrics']:
            html += '<div class="section"><h3>🏥 Métricas Clínicas</h3>'
            for metric, data in report['clinical_metrics'].items():
                if isinstance(data, dict) and 'value' in data:
                    html += f"""
                        <div class="metric">
                            <strong>{metric.replace('_', ' ').title()}:</strong> {data['value']:.3f}<br>
                            <em>{data.get('interpretation', '')}</em>
                        </div>
                    """
            html += '</div>'

        # Evaluación clínica
        if 'clinical_assessment' in report:
            assessment = report['clinical_assessment']
            html += f"""
                <div class="section">
                    <h3>🔍 Evaluación Clínica</h3>
                    <div class="{'success' if 'excelente' in assessment['overall_assessment'].lower() else 'warning' if 'aceptable' in assessment['overall_assessment'].lower() else 'error'}">
                        <strong>Evaluación General:</strong> {assessment['overall_assessment']}
                    </div>
                    <div class="success" if assessment['clinical_readiness'] else 'warning'}">
                        <strong>Preparación Clínica:</strong> {assessment['clinical_readiness']}
                    </div>
                </div>
            """

        # Recomendaciones
        if 'recommendations' in report:
            html += '<div class="section"><h3>💡 Recomendaciones</h3>'
            for category, items in report['recommendations'].items():
                if items:
                    html += f'<h4>{category.replace("_", " ").title()}</h4><ul>'
                    for item in items:
                        html += f'<li>{item}</li>'
                    html += '</ul>'
            html += '</div>'

        html += """
            </body>
            </html>
        """

        return html

    def create_executive_summary(self, report):
        """Crear resumen ejecutivo del reporte"""

        print("\n" + "="*80)
        print("📋 REPORTE CLÍNICO - RESUMEN EJECUTIVO")
        print("="*80)

        # Metadatos
        metadata = report['report_metadata']
        print(f"📅 Fecha de Evaluación: {metadata['generation_date'][:10]}")
        print(f"🎯 Caso Clínico: {metadata['case_type'].replace('_', ' ').title()}")
        print(f"🤖 Modelo Evaluado: {metadata['model_name']}")
        print(f"📊 Muestras Analizadas: {metadata['evaluation_samples']}")
        print(f"🏷️ Clases: {', '.join(metadata['classes_evaluated'])}")

        # Performance
        perf = report['performance_summary']
        print(f"\n📈 RENDIMIENTO GENERAL:")
        print(f"  Precisión: {perf['overall_accuracy']:.1%} | Grado: {perf['performance_grade']}")
        print(f"  F1-Score: {perf['macro_f1_score']:.3f} | Balance Precision-Recall: {'✅' if perf['macro_f1_score'] >= 0.8 else '⚠️'}")

        # Métricas clínicas clave
        if report['clinical_metrics']:
            print(f"\n🏥 MÉTRICAS CLÍNICAS CLAVE:")
            for metric, data in report['clinical_metrics'].items():
                if isinstance(data, dict) and 'value' in data:
                    status = "✅" if data.get('meets_standard', True) else "⚠️"
                    print(f"  {metric.replace('_', ' ').title()}: {data['value']:.1%} {status}")

        # Confiabilidad
        reliability = report['reliability_analysis']
        print(f"\n🔍 ANÁLISIS DE CONFIABILIDAD:")
        print(f"  Puntuación: {reliability['reliability_score']['score']:.3f} (Grado {reliability['reliability_score']['grade']})")
        print(f"  Tasa de Error: {reliability['error_rate']:.1f}%")
        print(f"  Casos de Baja Confianza: {reliability['low_confidence_percentage']:.1f}%")

        if reliability['critical_cases'] > 0:
            print(f"  🚨 Casos Críticos: {reliability['critical_cases']}")

        # Evaluación clínica
        if 'clinical_assessment' in report:
            assessment = report['clinical_assessment']
            print(f"\n🎯 EVALUACIÓN CLÍNICA:")
            print(f"  Estado: {assessment['clinical_readiness']}")

            if assessment['strengths']:
                print("  ✅ Fortalezas:")
                for strength in assessment['strengths'][:3]:
                    print(f"    • {strength}")

            if assessment['weaknesses']:
                print("  ⚠️ Debilidades:")
                for weakness in assessment['weaknesses'][:3]:
                    print(f"    • {weakness}")

        # Recomendaciones principales
        if 'recommendations' in report:
            recs = report['recommendations']
            print(f"\n💡 ACCIONES PRIORITARIAS:")

            # Mostrar acciones inmediatas
            if recs['immediate_actions']:
                for action in recs['immediate_actions'][:2]:
                    print(f"  🔴 {action}")

            # Mostrar mejoras del modelo
            if recs['model_improvements']:
                for improvement in recs['model_improvements'][:2]:
                    print(f"  🔧 {improvement}")

        print("\n" + "="*80)
        print("🎉 RESUMEN EJECUTIVO COMPLETADO")
        print("="*80)

# Generar reporte clínico completo
print("\n📄 Generando reporte clínico detallado...")

report_generator = ClinicalReportGenerator(SELECTED_CASE, case_config['classes'])

clinical_report = report_generator.generate_clinical_report(
    evaluation_results,
    model_metadata or {},
    difficult_cases_analysis
)

# Guardar reportes en múltiples formatos
json_filepath = report_generator.save_report_json(clinical_report)
html_filepath = report_generator.save_report_html(clinical_report)

print(f"✅ Reporte JSON guardado en: {json_filepath}")
print(f"✅ Reporte HTML guardado en: {html_filepath}")

# Mostrar resumen ejecutivo
report_generator.create_executive_summary(clinical_report)

#@title ## 💾 Preparación Final para Notebook 05

def prepare_comprehensive_final_config():
    """Preparar configuración completa para el notebook final"""

    # Recopilar todos los archivos generados
    generated_files = []
    visualization_files = []

    # Buscar archivos generados
    viz_dir = f'{RESULTS_PATH}/visualizations'
    if os.path.exists(viz_dir):
        visualization_files = [
            os.path.join(viz_dir, f) for f in os.listdir(viz_dir)
            if f.endswith(('.html', '.png', '.jpg', '.pdf'))
        ]

    reports_dir = f'{RESULTS_PATH}/reports'
    if os.path.exists(reports_dir):
        report_files = [
            os.path.join(reports_dir, f) for f in os.listdir(reports_dir)
            if f.endswith(('.json', '.html', '.pdf'))
        ]
    else:
        report_files = []

    # Calcular métricas de calidad
    quality_metrics = {
        'accuracy_score': evaluation_results['metrics']['accuracy'],
        'f1_score': evaluation_results['metrics']['f1_macro'],
        'reliability_score': clinical_report['reliability_analysis']['reliability_score']['score'],
        'error_rate': clinical_report['reliability_analysis']['error_rate'],
        'low_confidence_rate': clinical_report['reliability_analysis']['low_confidence_percentage']
    }

    # Determinar preparación para producción
    production_ready = (
        quality_metrics['accuracy_score'] >= 0.85 and
        quality_metrics['reliability_score'] >= 0.8 and
        quality_metrics['error_rate'] <= 15.0 and
        len(difficult_cases_analysis.get('critical_cases', [])) == 0
    )

    final_config = {
        'project_metadata': {
            'project_root': PROJECT_ROOT,
            'selected_case': SELECTED_CASE,
            'case_config': case_config,
            'evaluation_completed': True,
            'evaluation_date': datetime.now().isoformat(),
            'notebook_version': '04_complete'
        },

        'model_information': {
            'best_model_path': best_model_info['path'] if best_model_info else None,
            'model_metadata': model_metadata or {},
            'model_architecture': best_model.name if hasattr(best_model, 'name') else 'Unknown',
            'total_parameters': best_model.count_params() if best_model else 0
        },

        'evaluation_results': {
            'total_test_samples': len(evaluation_results['y_true']),
            'quality_metrics': quality_metrics,
            'class_performance': {
                class_name: {
                    'precision': evaluation_results['metrics'][f'precision_{class_name}'],
                    'recall': evaluation_results['metrics'][f'recall_{class_name}'],
                    'f1_score': evaluation_results['metrics'][f'f1_{class_name}']
                }
                for class_name in case_config['classes']
            },
            'clinical_metrics': clinical_report['clinical_metrics'],
            'performance_grade': clinical_report['performance_summary']['performance_grade']
        },

        'difficulty_analysis': {
            'total_difficult_cases': len(difficult_cases_analysis['low_confidence_cases']),
            'critical_cases': len(difficult_cases_analysis.get('critical_cases', [])),
            'error_patterns': difficult_cases_analysis.get('error_patterns', {}),
            'main_challenges': difficult_cases_analysis.get('recommendations', [])
        },

        'generated_artifacts': {
            'visualizations': visualization_files,
            'reports': report_files,
            'clinical_report_json': json_filepath,
            'clinical_report_html': html_filepath
        },

        'clinical_assessment': clinical_report['clinical_assessment'],

        'production_readiness': {
            'ready_for_production': production_ready,
            'readiness_score': (
                quality_metrics['accuracy_score'] * 0.3 +
                quality_metrics['f1_score'] * 0.3 +
                quality_metrics['reliability_score'] * 0.4
            ),
            'blocking_issues': [],
            'recommendations': clinical_report['recommendations']
        },

        'next_notebook_tasks': [
            'Revisar y validar todos los resultados de evaluación',
            'Implementar mejoras recomendadas del modelo',
            'Realizar validación cruzada adicional si es necesario',
            'Preparar modelo para exportación y despliegue',
            'Generar documentación final del proyecto',
            'Crear pipeline de monitoreo post-despliegue'
        ],

        'quality_gates': {
            'minimum_accuracy': 0.80,
            'minimum_f1_score': 0.75,
            'maximum_error_rate': 20.0,
            'maximum_critical_cases': 5,
            'gates_passed': {
                'accuracy_gate': quality_metrics['accuracy_score'] >= 0.80,
                'f1_gate': quality_metrics['f1_score'] >= 0.75,
                'error_rate_gate': quality_metrics['error_rate'] <= 20.0,
                'critical_cases_gate': len(difficult_cases_analysis.get('critical_cases', [])) <= 5
            }
        }
    }

    # Agregar issues bloqueantes si existen
    if not final_config['quality_gates']['gates_passed']['accuracy_gate']:
        final_config['production_readiness']['blocking_issues'].append(
            f"Precisión {quality_metrics['accuracy_score']:.3f} por debajo del mínimo requerido (0.80)"
        )

    if not final_config['quality_gates']['gates_passed']['critical_cases_gate']:
        final_config['production_readiness']['blocking_issues'].append(
            f"Se detectaron {len(difficult_cases_analysis.get('critical_cases', []))} casos críticos"
        )

    return final_config

# Preparar configuración final completa
print("🔄 Preparando configuración final para Notebook 05...")

final_config = prepare_comprehensive_final_config()

# Guardar configuración
config_output_file = f'{PROJECT_ROOT}/config/notebook_04_output.json'
os.makedirs(os.path.dirname(config_output_file), exist_ok=True)

with open(config_output_file, 'w', encoding='utf-8') as f:
    json.dump(final_config, f, indent=2, ensure_ascii=False, default=str)

# Mostrar resumen final
print("\n" + "🚀" + "="*78 + "🚀")
print("   NOTEBOOK 04 - EVALUACIÓN E INTERPRETACIÓN COMPLETADO")
print("🚀" + "="*78 + "🚀")

print(f"\n📊 RESULTADOS FINALES DE EVALUACIÓN:")
print(f"✅ Muestras evaluadas: {len(evaluation_results['y_true'])}")
print(f"🎯 Precisión alcanzada: {evaluation_results['metrics']['accuracy']:.1%}")
print(f"📈 F1-Score macro: {evaluation_results['metrics']['f1_macro']:.3f}")
print(f"🏆 Grado de rendimiento: {clinical_report['performance_summary']['performance_grade']}")
print(f"🔍 Puntuación de confiabilidad: {clinical_report['reliability_analysis']['reliability_score']['score']:.3f}")

print(f"\n🎯 ESTADO DE PREPARACIÓN PARA PRODUCCIÓN:")
if final_config['production_readiness']['ready_for_production']:
    print("✅ MODELO LISTO para consideración de despliegue clínico")
else:
    print("⚠️ MODELO REQUIERE mejoras antes del despliegue")

if final_config['production_readiness']['blocking_issues']:
    print("🚫 Issues bloqueantes:")
    for issue in final_config['production_readiness']['blocking_issues']:
        print(f"  • {issue}")

print(f"\n📁 ARCHIVOS GENERADOS:")
print(f"✅ Configuración final: {config_output_file}")
print(f"✅ Reporte clínico JSON: {json_filepath}")
print(f"✅ Reporte clínico HTML: {html_filepath}")
print(f"✅ Visualizaciones: {len(final_config['generated_artifacts']['visualizations'])} archivos")

print(f"\n🎯 PRÓXIMO PASO:")
print("📋 Ejecutar Notebook 05 - Ajustes, Post-entrenamiento y Despliegue")
print("🔧 Este notebook implementará mejoras finales y preparará el despliegue")

print(f"\n📋 CHECKLIST DE EVALUACIÓN COMPLETADO:")
print("✅ Modelo evaluado exhaustivamente en conjunto de test")
print("✅ Métricas clínicas detalladas calculadas")
print("✅ Interpretabilidad implementada con Grad-CAM")
print("✅ Casos difíciles identificados y analizados")
print("✅ Reporte clínico profesional generado")
print("✅ Visualizaciones interactivas creadas")
print("✅ Análisis de confianza y errores completado")
print("✅ Recomendaciones específicas documentadas")
print("✅ Preparación para producción evaluada")

print("\n" + "🎉" + "="*78 + "🎉")
print("   ¡EVALUACIÓN CLÍNICA COMPLETADA EXITOSAMENTE!")
print("🎉" + "="*78 + "🎉")

# Logging final
logging.info("Notebook 04 completado exitosamente")
logging.info(f"Precisión final: {evaluation_results['metrics']['accuracy']:.3f}")
logging.info(f"F1-Score final: {evaluation_results['metrics']['f1_macro']:.3f}")
logging.info(f"Casos problemáticos: {len(difficult_cases_analysis['low_confidence_cases']) + len(difficult_cases_analysis['incorrect_cases'])}")
logging.info(f"Archivos generados: {len(final_config['generated_artifacts']['visualizations']) + len(final_config['generated_artifacts']['reports'])}")

print("\n💡 TIP: Revisa el reporte HTML generado para un análisis visual completo de los resultados")
print(f"🌐 Abrir: {html_filepath}")